In [149]:
from enum import Enum
import re
from pathlib import Path

class Kind(Enum):
    HIGH_CARD = 0
    ONE_PAIR = 1
    TWO_PAIRS = 2
    THREE = 3
    FULL_HOUSE = 4
    FOUR = 5
    FIVE = 6

hand_map = { "2": 1, "3": 2, "4": 3, "5": 4, "6": 5, "7": 6,
            "8": 7, "9": 8, "T": 9, "J": 10, "Q": 11, "K": 12, "A": 13}


class Hand:

    def __init__(self, hand):
        self.hand = [hand_map[c] for c in hand]
        mc = sorted((v for _, v in Counter(hand).most_common(2)), reverse=True)
        if mc == [5]:
            self.kind = Kind.FIVE
        elif mc[0] == 4:
            self.kind = Kind.FOUR
        elif mc == [3, 2]:
            self.kind = Kind.FULL_HOUSE
        elif mc[0] == 3:
            self.kind = Kind.THREE
        elif mc == [2, 2]:
            self.kind = Kind.TWO_PAIRS
        elif mc[0] == 2:
            self.kind = Kind.ONE_PAIR
        else:
            self.kind = Kind.HIGH_CARD

    def __lt__(self, card):
        if self.kind == card.kind:
            return self.hand < card.hand
        return self.kind.value < card.kind.value

    def __repr__(self):
        return f"Hand<{self.hand}->{self.kind}>"

hand2_map = {**hand_map, **{"J": 0}}

class Hand2:

    def __init__(self, hand):
        self.hand = [hand2_map[c] for c in hand]
        counter = Counter(hand)
        js = 0
        if "J" in counter:
            js = counter["J"]
            del counter["J"]
        mc = sorted((v for _, v in counter.most_common(2)), reverse=True)

        if not mc or mc == [5 - js]:
            self.kind = Kind.FIVE
        elif mc[0] == 4 - js:
            self.kind = Kind.FOUR
        elif sum(mc) == 5 - js:
            self.kind = Kind.FULL_HOUSE
        elif mc[0] == 3 - js:
            self.kind = Kind.THREE
        elif sum(mc) == 4 - js:
            self.kind = Kind.TWO_PAIRS
        elif mc[0] == 2 - js:
            self.kind = Kind.ONE_PAIR
        else:
            self.kind = Kind.HIGH_CARD

    def __lt__(self, card):
        if self.kind == card.kind:
            return self.hand < card.hand
        return self.kind.value < card.kind.value

    def __repr__(self):
        return f"Hand<{self.hand}->{self.kind}>"


def load_file(path: str, cls):
    res = []
    with Path(path).open() as f:
        for line in f.readlines():
            hand, bid = line.strip().split(" ")
            res.append((cls(hand), int(bid)))
    return res

In [150]:
assert sum(rank * bid for rank, (_, bid) in enumerate(sorted(load_file("./07_test.txt", Hand)), 1)) == 6440

In [151]:
sum(rank * bid for rank, (_, bid) in enumerate(sorted(load_file("./07_input.txt", Hand)), 1))  # 250120186

250120186

In [152]:
assert sum(rank * bid for rank, (_, bid) in enumerate(sorted(load_file("./07_test.txt", Hand2)), 1)) == 5905

In [153]:
sum(rank * bid for rank, (_, bid) in enumerate(sorted(load_file("./07_input.txt", Hand2)), 1))  # 250665248

250665248